In [ ]:
import cv2
import numpy as np

import cv2

cap = cv2.VideoCapture("C:/Users/ygl/Desktop/tempt_projectC-main/userVideos/hankug/carVideoTest.mp4")
# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2()
if cap.isOpened():
    print("open")
else:
    print("fail")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    # 1. Object Detection
    mask = object_detector.apply(frame)

    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 5000 and area < 10000:
             #cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
             x, y, w, h = cv2.boundingRect(cnt)
             print(x, y, w, h)
             cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
             #cv2.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
    cv2.imshow("car_video_test", frame)

cap.release()
cv2.destroyAllWindows()



def get_bbox_image(bbox,frame):
  return frame[bbox[0]:bbox[1],bbox[2]:bbox[3]].copy()

def get_bboxs_images(past_bboxs,frames):
  return [[get_bbox_image(bbox,frame) for frame in frames]for bbox in past_bboxs]


def eliminate_objects(remain_bbox,eliminate_bboxs,frame):
  frame = frame.copy()



def detect_bbox(thresh,max_diff,size,frames):
  frames = [cv2.cvtColor(cv2.resize(frame, cv2.COLOR_BGR2GRAY), dsize=(size, size), interpolation=cv2.INTER_AREA) for frame in frames]
  frames = [cv2.absdiff(frames[num-1],frame[num]) for num in range(1,len(frames)) ]
  frames = [cv2.threshold(frame, thresh, 255, cv2.THRESH_BINARY)[1] for frame in frames]
  frames = [cv2.bitwise_and(frames[num-1], frame[num]) for num in range(1,len(frames))]
  k = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
  frames = [cv2.morphologyEx(frame, cv2.MORPH_OPEN, k) for frame in frames]
  nonzeros = [for frame in frames np.nonzero(diff) if cv2.countNonZero(frame) > max_diff else pass]
  bboxs = [[min(nzero[1]), min(nzero[0]),max(nzero[1]), max(nzero[0])] for nzero in nonzeros]

  return bboxs

def predict_bbox():


 
thresh = 25
max_diff = 5
 
a, b, c = None, None, None
 
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 320)
 
if cap.isOpened():
    ret, a = cap.read()
    ret, b = cap.read()
    while ret:
        ret, c = cap.read()
        draw = c.copy()
        if not ret:
            break
 
        a_gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
        b_gray = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
        c_gray = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)
 
        diff1 = cv2.absdiff(a_gray, b_gray)
        diff2 = cv2.absdiff(b_gray, c_gray)
 
        ret, diff1_t = cv2.threshold(diff1, thresh, 255, cv2.THRESH_BINARY)
        ret, diff2_t = cv2.threshold(diff2, thresh, 255, cv2.THRESH_BINARY)
 
        diff = cv2.bitwise_and(diff1_t, diff2_t)
 
        k = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
        diff = cv2.morphologyEx(diff, cv2.MORPH_OPEN, k)
 
        diff_cnt = cv2.countNonZero(diff)
        if diff_cnt > max_diff:
            nzero = np.nonzero(diff)
            cv2.rectangle(draw, (min(nzero[1]), min(nzero[0])),
                          (max(nzero[1]), max(nzero[0])), (0, 255, 0), 2)
 
            '''
            rectangle: pt1, pt2 기준으로 사각형 프레임을 만들어줌.
            nzero: diff는 카메라 영상과 사이즈가 같으며, a, b프레임의 차이 어레이를 의미함.
            (min(nzero[1]), min(nzero[0]): diff에서 0이 아닌 값 중 행, 열이 가장 작은 포인트
            (max(nzero[1]), max(nzero[0]): diff에서 0이 아닌 값 중 행, 열이 가장 큰 포인트
            (0, 255, 0): 사각형을 그릴 색상 값
            2 : thickness
            '''
 
            cv2.putText(draw, "Motion detected!!", (10, 30),
                        cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255))
 
        stacked = np.hstack((draw, cv2.cvtColor(diff, cv2.COLOR_GRAY2BGR)))
        cv2.imshow('motion', stacked)
 
        a = b
        b = c
 
        if cv2.waitKey(1) & 0xFF == 27:
            break